# Training & Generation

For training we use [OpenNMT-py](http://opennmt.net/OpenNMT-py/) which is based on PyTorch. This is a run-of-the mill Seq2Seq library that is traditionally used for neural machine translation.

I have forked their repository and adapted the code to incorporate ideas from [Stay On-Topic: Generating
Context-specific Fake Restaurant Reviews](https://arxiv.org/pdf/1805.02400.pdf)

Namely, during generation (beam-search) some words of the vocabulary will be randomly penalized, on top of that, recurring words will be penalized as well. I've also adapted the `-ignore_when_blocking` parameter from OpenNMT to reduce the penalty for certain 'anchor' words. Unlike the paper, this reduced penalty is only active after the first step and not for step 0.

Unlike the paper quoted above, I also did not add an obfuscation step.

My fork can be found under:

https://github.com/hokkaido/OpenNMT-py/tree/translate-improvements

The changes can be found at https://github.com/hokkaido/OpenNMT-py/pull/1/files

## Training

OpenNMT requires a preprocessing step, we limit the vocabulary size by specifying `-tgt_words_min_frequency`

In [ ]:
!python preprocess.py -train_src data/train_context.txt -train_tgt data/train_reviews.txt -valid_src data/val_context.txt -valid_tgt data/val_reviews.txt -save_data models/model15k -tgt_words_min_frequency 10

After that we can train the model as follows:

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train.py --data models/model15k --save_model models/model15k750k --train_steps 750000 --valid_steps 93750 --learning_rate_decay 0.5 --optim adam --learning_rate 0.001 --start_decay_steps 281250 --world_size 1 --gpu_ranks 0 --batch_size 64 --valid_batch_size 16

I had to reduce the validation batch size to 16 because of memory issues. The training took 17 hours. However, the process could have been stopped at an earlier time. Perplexity and accuracy plateaued early on, and subjectivally decent fake review generation results could be achieved with training times of only 2 hours.

Below you find some excerpts of the training log:

    [2019-06-12 09:05:51,800 INFO] Step 468700/750000; acc:  35.11; ppl: 24.87; xent: 3.21; lr: 0.00000; 11100/27825 tok/s;  38659 sec
    [2019-06-12 09:05:55,828 INFO] Step 468750/750000; acc:  34.51; ppl: 26.15; xent: 3.26; lr: 0.00000; 11981/27910 tok/s;  38663 sec
    [2019-06-12 09:05:55,829 INFO] Loading dataset from models/model15k.valid.0.pt
    [2019-06-12 09:05:56,043 INFO] number of examples: 15000
    [2019-06-12 09:09:38,355 INFO] Validation perplexity: 93.3477
    [2019-06-12 09:09:38,355 INFO] Validation accuracy: 24.1794
    
    ...

    [2019-06-12 15:34:42,982 INFO] Step 749850/750000; acc:  34.66; ppl: 25.92; xent: 3.26; lr: 0.00000; 11283/28185 tok/s;  61990 sec
    [2019-06-12 15:34:46,808 INFO] Step 749900/750000; acc:  34.88; ppl: 25.52; xent: 3.24; lr: 0.00000; 11560/28148 tok/s;  61994 sec
    [2019-06-12 15:34:50,988 INFO] Step 749950/750000; acc:  34.53; ppl: 25.97; xent: 3.26; lr: 0.00000; 10780/28526 tok/s;  61998 sec
    [2019-06-12 15:34:55,011 INFO] Step 750000/750000; acc:  34.82; ppl: 25.71; xent: 3.25; lr: 0.00000; 10835/28475 tok/s;  62002 sec
    [2019-06-12 15:34:55,012 INFO] Loading dataset from models/model15k.valid.0.pt
    [2019-06-12 15:34:55,229 INFO] number of examples: 15000
    [2019-06-12 15:38:36,945 INFO] Validation perplexity: 93.3476
    [2019-06-12 15:38:36,946 INFO] Validation accuracy: 24.1794
    [2019-06-12 15:38:37,472 INFO] Saving checkpoint models/model15k750k_step_750000.pt

## Fake review generation

To generate fake reviews, we can use the trained model and our adapted OpenNMT-py library. As a demonstration, below is the output of 40 random sentences picked from the test set. The model seems to be able to infer context specific knowledge such as places or categories and the appropriate sentiment - there are only a few incoherent  sentences or phrases.

An anectodal review with my friends seems to confirm that the reviews are of quite high quality and difficult to detect as fake.

Integrating more contextual information (business name or similar) into the training examples would probably improve the variety of the output.

In [13]:
!python ../OpenNMT-py/translate.py -model ../OpenNMT-py/models/model15k750k_step_750000.pt -src test_context_excerpt.txt -replace_unk -verbose -max_length 50 -ignore_when_blocking "." "" "" "," "i" "thus" "our" "them" "they" "their" "them" "us" "we" "ours" "theirs" "she" "her" "hers" "him" "his" "my" "mine" "but" "and" "or" "/" ":" "it" "its" "if"  --beam_size 5


instantiated
[2019-06-15 17:39:59,979 INFO] Translating shard 0.
init beamsearch

SENT 1: ['4.0', 'las', 'vegas', 'restaurants', ',', 'vietnamese']
PRED 1: i love this place ! the food is always fresh and delicious . it 's a little pricey , but worth every penny .
PRED SCORE: -33.2517

SENT 2: ['1.0', 'calgary', 'asian', 'fusion', ',', 'chicken', 'wings', ',', 'food', ',', 'cafes', ',', 'chinese', ',', 'desserts', ',', 'juice', 'bars', '&', 'smoothies', ',', 'restaurants']
PRED 2: this is the worst bubble tea place i 've ever been to . it 's so expensive and they do n't have a lot of options .
PRED SCORE: -36.3755

SENT 3: ['3.0', 'scottsdale', 'japanese', ',', 'nightlife', ',', 'restaurants', ',', 'sushi', 'bars']
PRED 3: this is a good place to go if you 're in the mood for japanese food . it 's not bad , but nothing special .
PRED SCORE: -36.7155

SENT 4: ['1.0', 'phoenix', 'sandwiches', ',', 'beer', ',', 'wine', '&', 'spirits', ',', 'beer', 'bar', ',', 'bars', ',', 'breakfast', '&'

We can now use our test set to generate 6000 fake reviews.

In [15]:
!python ../OpenNMT-py/translate.py -model ../OpenNMT-py/models/model15k750k_step_750000.pt -src ../OpenNMT-py/data/test_context.txt -replace_unk -verbose -max_length 50 -ignore_when_blocking "." "" "" "," "i" "thus" "our" "them" "they" "their" "them" "us" "we" "ours" "theirs" "she" "her" "hers" "him" "his" "my" "mine" "but" "and" "or" "/" ":" "it" "its" "if"  --beam_size 5 --output fake-reviews/generated.txt

instantiated
[2019-06-15 17:49:12,882 INFO] Translating shard 0.
init beamsearch

SENT 1: ['5.0', 'tempe', 'shopping', ',', 'home', '&', 'garden', ',', 'hardware', 'stores']
PRED 1: this is the best hardware store i 've ever been to . they have everything you need for your home , and their prices are very reasonable !
PRED SCORE: -34.6563

SENT 2: ['4.0', 'champaign', 'cupcakes', ',', 'ice', 'cream', '&', 'frozen', 'yogurt', ',', 'event', 'planning', '&', 'services', ',', 'food', ',', 'coffee', '&', 'tea', ',', 'caterers', ',', 'bakeries']
PRED 2: this place is a little pricey but the cupcakes are really good . i love that they have gluten free options as well !
PRED SCORE: -33.8857

SENT 3: ['4.0', 'las', 'vegas', 'shopping', 'centers', ',', 'resorts', ',', 'arts', '&', 'entertainment', ',', 'hotels', ',', 'shopping', ',', 'event', 'planning', '&', 'services', ',', 'automotive', ',', 'restaurants', ',', 'hotels', '&', 'travel', ',', 'casinos', ',', 'car', 'wash']
PRED 3: this is a gre


SENT 31: ['5.0', 'phoenix', 'seafood', ',', 'sushi', 'bars', ',', 'restaurants', ',', 'japanese']
PRED 31: this place is amazing ! the sushi was so fresh and delicious . i will definitely be back to try more of their menu .
PRED SCORE: -32.9910

SENT 32: ['5.0', 'las', 'vegas', 'bars', ',', 'dim', 'sum', ',', 'japanese', ',', 'asian', 'fusion', ',', 'restaurants', ',', 'chinese', ',', 'breakfast', '&', 'brunch', ',', 'nightlife']
PRED 32: the best sushi i have ever had in my life . great service and amazing food !
PRED SCORE: -25.4140

SENT 33: ['5.0', 'madison', 'restaurants', ',', 'vegetarian', ',', 'diners', ',', 'american', '-lrb-', 'traditional', '-rrb-']
PRED 33: one of my favorite places to eat in madison . the food is always delicious and i love that they have vegan options !
PRED SCORE: -28.9015

SENT 34: ['3.0', 'toronto', 'restaurants', ',', 'canadian', '-lrb-', 'new', '-rrb-', ',', 'greek']
PRED 34: this place is good for a quick bite to eat . it 's not the best greek food


SENT 61: ['1.0', 'toronto', 'sushi', 'bars', ',', 'italian', ',', 'thai', ',', 'restaurants']
PRED 61: worst sushi i 've ever had in my life . do n't waste your time or money here !
PRED SCORE: -22.9143

SENT 62: ['3.0', 'las', 'vegas', 'lounges', ',', 'nightlife', ',', 'arts', '&', 'entertainment', ',', 'arcades', ',', 'bars']
PRED 62: it 's a pretty cool place to hang out with friends . the drinks are expensive and they do n't take credit cards .
PRED SCORE: -35.3436

SENT 63: ['4.0', 'las', 'vegas', 'outlet', 'stores', ',', 'restaurants', ',', 'shopping', 'centers', ',', 'fashion', ',', 'shopping']
PRED 63: it 's a nice outlet mall . i like the variety of stores that you can choose from .
PRED SCORE: -30.4521

SENT 64: ['5.0', 'charlotte', 'religious', 'organizations', ',', 'public', 'services', '&', 'government', ',', 'arts', '&', 'entertainment', ',', 'museums', ',', 'churches', ',', 'tours', ',', 'historical', 'tours', ',', 'libraries', ',', 'hotels', '&', 'travel']
PRED 64: thi


SENT 91: ['5.0', 'toronto', 'coffee', '&', 'tea', ',', 'food']
PRED 91: this is the best coffee shop i 've been to in toronto . they have a great selection of coffees , teas and pastries .
PRED SCORE: -34.2456

SENT 92: ['3.0', 'phoenix', 'restaurants', ',', 'food', ',', 'greek']
PRED 92: the food was ok , nothing special . i would n't go out of my way to eat here again .
PRED SCORE: -25.1583

SENT 93: ['3.0', 'las', 'vegas', 'cafes', ',', 'american', '-lrb-', 'new', '-rrb-', ',', 'restaurants']
PRED 93: the food was good , but i would n't go out of my way to eat here again .
PRED SCORE: -20.1484

SENT 94: ['2.0', 'phoenix', 'allergists', ',', 'sleep', 'specialists', ',', 'cosmetic', 'surgeons', ',', 'dentists', ',', 'ear', 'nose', '&', 'throat', ',', 'health', '&', 'medical', ',', 'cosmetic', 'dentists', ',', 'doctors', ',', 'audiologist']
PRED 94: dr. phoenix and his staff are very nice . however , the billing department is a complete rip off !
PRED SCORE: -31.0578

SENT 95: ['1.0',


SENT 151: ['3.0', 'peoria', 'restaurants', ',', 'american', '-lrb-', 'traditional', '-rrb-', ',', 'steakhouses', ',', 'barbeque']
PRED 151: the food was good , but i would n't go out of my way to eat here again .
PRED SCORE: -21.3472

SENT 152: ['4.0', 'las', 'vegas', 'shopping', 'centers', ',', 'resorts', ',', 'arts', '&', 'entertainment', ',', 'hotels', ',', 'shopping', ',', 'event', 'planning', '&', 'services', ',', 'automotive', ',', 'restaurants', ',', 'hotels', '&', 'travel', ',', 'casinos', ',', 'car', 'wash']
PRED 152: this hotel is beautiful and the rooms are very nice . i would definitely stay here again !
PRED SCORE: -25.4836

SENT 153: ['1.0', 'goodyear', 'gastroenterologist', ',', 'family', 'practice', ',', 'doctors', ',', 'health', '&', 'medical']
PRED 153: this is the worst office i have ever been to . they do n't care about their patients and are very rude .
PRED SCORE: -30.1324

SENT 154: ['5.0', 'henderson', 'electricians', ',', 'lighting', 'fixtures', '&', 'equipmen

...

init beamsearch

SENT 5791: ['5.0', 'phoenix', 'nail', 'salons', ',', 'waxing', ',', 'beauty', '&', 'spas', ',', 'hair', 'removal']
PRED 5791: this is the best nail salon i have ever been to . they are very friendly and do a great job !
PRED SCORE: -23.3667

SENT 5792: ['4.0', 'markham', 'japanese', ',', 'sushi', 'bars', ',', 'restaurants']
PRED 5792: i 've been coming here for years . the food is always fresh and tasty , but it 's a bit pricey .
PRED SCORE: -34.5242

SENT 5793: ['1.0', 'phoenix', 'hotels', '&', 'travel', ',', 'event', 'planning', '&', 'services', ',', 'hotels']
PRED 5793: this is the worst hotel i have ever stayed at . they do n't care about their customers and are very rude .
PRED SCORE: -31.5359

SENT 5794: ['2.0', 'pittsburgh', 'breakfast', '&', 'brunch', ',', 'american', '-lrb-', 'new', '-rrb-', ',', 'cocktail', 'bars', ',', 'bars', ',', 'nightlife', ',', 'restaurants', ',', 'american', '-lrb-', 'traditional', '-rrb-']
PRED 5794: food was good , but not worth the 


SENT 5821: ['5.0', 'las', 'vegas', 'general', 'litigation', ',', 'workers', 'compensation', 'law', ',', 'legal', 'services', ',', 'lawyers', ',', 'professional', 'services', ',', 'personal', 'injury', 'law']
PRED 5821: this is the best law firm i have ever dealt with . they are very professional and courteous .
PRED SCORE: -24.9302

SENT 5822: ['1.0', 'las', 'vegas', 'nail', 'salons', ',', 'beauty', '&', 'spas']
PRED 5822: worst nail salon i 've ever been to . they did a horrible job on my nails and the price was very high for what you get
PRED SCORE: -36.5555

SENT 5823: ['5.0', 'las', 'vegas', 'sandwiches', ',', 'burgers', ',', 'restaurants', ',', 'american', '-lrb-', 'traditional', '-rrb-', ',', 'salad']
PRED 5823: this is the best burger place i 've ever been to . it 's a must try if you 're in vegas !
PRED SCORE: -26.4958

SENT 5824: ['5.0', 'beachwood', 'american', '-lrb-', 'new', '-rrb-', ',', 'health', '&', 'medical', ',', 'doctors', ',', 'bars', ',', 'restaurants', ',', 'opht

init beamsearch

SENT 5881: ['4.0', 'henderson', 'thai', ',', 'restaurants']
PRED 5881: very good thai food . i had the yellow curry with chicken and it was delicious !
PRED SCORE: -22.6338

SENT 5882: ['5.0', 'scottsdale', 'body', 'shops', ',', 'towing', ',', 'automotive']
PRED 5882: i have used this company for a few years now . they are always on time , professional and very friendly !
PRED SCORE: -31.6741

SENT 5883: ['5.0', 'phoenix', 'seafood', ',', 'restaurants', ',', 'thai', ',', 'noodles', ',', 'asian', 'fusion']
PRED 5883: the best thai food i 've had in phoenix . it 's a little pricey but worth every penny !
PRED SCORE: -27.4760

SENT 5884: ['5.0', 'cleveland', 'health', '&', 'medical', ',', 'ophthalmologists', ',', 'internal', 'medicine', ',', 'cardiologists', ',', 'hospitals', ',', 'optometrists', ',', 'medical', 'centers', ',', 'doctors']
PRED 5884: i 've been going to dr. cleveland for a few years now . he is the best !
PRED SCORE: -21.3107

SENT 5885: ['5.0', 'chandler'

init beamsearch

SENT 5941: ['4.0', 'las', 'vegas', 'italian', ',', 'restaurants', ',', 'ice', 'cream', '&', 'frozen', 'yogurt', ',', 'pizza', ',', 'american', '-lrb-', 'traditional', '-rrb-', ',', 'food', 'stands', ',', 'food']
PRED 5941: i had a great time at trevi . the food was good , but nothing special . it 's pricey for what you get and there is not much to choose from !
PRED SCORE: -56.2058

SENT 5942: ['1.0', 'pittsburgh', 'mobile', 'phones', ',', 'it', 'services', '&', 'computer', 'repair', ',', 'electronics', ',', 'telecommunications', ',', 'shopping', ',', 'mobile', 'phone', 'accessories', ',', 'local', 'services']
PRED 5942: do n't waste your time or money on this store . the employees are rude and not helpful at all .
PRED SCORE: -29.7984

SENT 5943: ['3.0', 'chandler', 'beauty', '&', 'spas', ',', 'hair', 'salons']
PRED 5943: i 've been going to this salon for a few years now . the staff is very friendly and they do an excellent job !
PRED SCORE: -29.5557

SENT 5944: ['5.


SENT 5971: ['3.0', 'las', 'vegas', 'hair', 'removal', ',', 'skin', 'care', ',', 'day', 'spas', ',', 'waxing', ',', 'beauty', '&', 'spas', ',', 'nail', 'salons']
PRED 5971: i 've been going to this place for a few years now . the staff is very friendly and they do an excellent job on my nails !
PRED SCORE: -33.5148

SENT 5972: ['4.0', 'toronto', 'music', '&', 'dvds', ',', 'shopping', ',', 'books', ',', 'mags', ',', 'music', '&', 'video', ',', 'bookstores']
PRED 5972: good place to buy books , cds and dvds . they also have a great selection of vinyl .
PRED SCORE: -30.7713

SENT 5973: ['5.0', 'las', 'vegas', 'nightlife', ',', 'cocktail', 'bars', ',', 'restaurants', ',', 'bars', ',', 'mexican']
PRED 5973: i love this place . the food is amazing and they have a great happy hour !
PRED SCORE: -23.8985

SENT 5974: ['1.0', 'las', 'vegas', 'plumbing', ',', 'home', 'services', ',', 'heating', '&', 'air', 'conditioning/hvac', ',', 'contractors', ',', 'hydro-jetting', ',', 'appliances', '&', 'rep